<a href="https://colab.research.google.com/github/MeenTers/Project/blob/main/Car_DMG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup 

In [1]:
import os
import sys
import torch
import numpy as np
import json
if torch.__version__=='1.6.0+cu101' and sys.platform.startswith('linux'):
    !pip install pytorch3d
else:
    need_pytorch3d=False
    try:
        import pytorch3d
    except ModuleNotFoundError:
        need_pytorch3d=True
    if need_pytorch3d:
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   118  100   118    0     0    280      0 --:--:-- --:--:-- --:--:--   280
100  404k    0  404k    0     0   679k      0 --:--:-- --:--:-- --:--:--  679k
  Cloning https://github.com/facebookresearch/pytorch3d.git (to revision stable) to /tmp/pip-req-build-mnt6978d
  Running command git clone -q https://github.com/facebookresearch/pytorch3d.git /tmp/pip-req-build-mnt6978d
  Running command git checkout -q 3fc3ea507bfbf7e1c2d5023a6db202698206f91d
     |████████████████████████████████| 55 kB 3.5 MB/s 
     |████████████████████████████████| 596 kB 65.7 MB/s 
  Created wheel for pytorch3d: filename=pytorch3d-0.6.1-cp37-cp37m-linux_x86_64.whl size=30527351 sha256=11f62fc4ebb3d1f82153fbd6dc2ddaab4abedfe7aee29a67ea1d8350b8653f32
  Stored in directory: /tmp/pip-ephem-wheel-cache-s5xgq5ne/wheels/4c/03/f0/326bb241eb86a155a12f217c81e16

In [2]:
!pip install utils

In [5]:
!pip install Params

In [3]:
import matplotlib.pyplot as plt
from skimage.io import imread
from utils import *

# Util function for loading meshes
from pytorch3d.io import load_obj

# Data structures and functions for rendering
from pytorch3d.structures import Meshes
from pytorch3d.renderer import (
    look_at_view_transform,
    FoVPerspectiveCameras, 
    FoVOrthographicCameras, 
    Materials, 
    RasterizationSettings, 
    MeshRenderer, 
    MeshRasterizer,  
    SoftPhongShader,
    TexturesVertex,
    TexturesAtlas,
    PointsRenderer,
    PointsRasterizationSettings,
    PointsRasterizer
)

In [4]:
# Set our device:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")

# Install TH SARABUN

In [14]:
!wget -q https://github.com/Phonbopit/sarabun-webfont/raw/master/fonts/thsarabunnew-webfont.ttf

In [15]:
# !pip install -U --pre matplotlib  
import matplotlib as mpl
mpl.font_manager.fontManager.addfont('thsarabunnew-webfont.ttf')
mpl.rc('font', family='TH Sarabun New')

# Import Functions

In [6]:
import torch
from torch import nn, load, utils
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from torchvision import datasets, transforms, models
from torchvision.models.vgg import model_urls
from os import path, listdir
model_urls['vgg19'] = model_urls['vgg19'].replace('https://', 'http://')

In [7]:
from scipy.spatial import distance_matrix
from scipy.spatial import distance
import pandas as pd
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2

In [71]:
from PIL import Image 
import PIL
import os

# VGG Model

In [9]:
class VGG:
	def __init__(self):
		model = models.vgg19(pretrained=True, progress=True)
		model.classifier = nn.Sequential(*list(model.classifier.children())[:3])
		self.model = model.cuda().eval()

	def __call__(self, x):
		return self.model(x)

In [10]:
vgg = VGG()

Downloading: "http://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

# Create Functions

In [11]:
def get_features(model, loader):
    features = []
    with torch.no_grad():
        for batch, _ in tqdm(loader):
            if torch.cuda.is_available():
                batch = batch.cuda()
            b_features = model(batch).detach().cpu().numpy()
            for f in b_features:
                features.append(f)

    return features

In [12]:
def get_dataset(images_path):
  transform = transforms.Compose([
    transforms.Resize(size=32),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
  ])

  dataset = datasets.ImageFolder(images_path, transform=transform)
  loader = utils.data.DataLoader(dataset, batch_size=100, shuffle=False, num_workers=1, pin_memory=True)
  return dataset, loader

In [16]:
def get_euclidean(base_car,diff_view):
    dist = []
    for i in range(len(diff_view)):
        x = distance.euclidean(base_car[i],diff_view[i])
        dist.append(x)
    return(dist)

In [94]:
def get_damage(distance):
    f_car = ((distance[0] - 3.57)/(37.564-3.57)*100)
    r_car = ((distance[1] - 4.308)/(37.966-4.308)*100)
    b_car = ((distance[2] - 3.401)/(21.090-3.401)*100)
    l_car = ((distance[3] - 11.230)/(48.075-11.230)*100)
    print(f'ด้านหน้าของรถยนต์มีความเสียหาย {round(f_car,2)}%')
    print(f'ด้านขวาของรถยนต์มีความเสียหาย {round(r_car,2)}%')
    print(f'ด้านหลังของรถยนต์มีความเสียหาย {round(b_car,2)}%')
    print(f'ด้านซ้ายของรถยนต์มีความเสียหาย {round(l_car,2)}%')

In [31]:
def tensor_to_image(tensor):
    tensor = tensor*255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor)>3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]
    return PIL.Image.fromarray(tensor)

In [84]:
def get_image_base(obj_filename,image_size,dist,device=device):
    verts, faces, aux = load_obj(
        obj_filename,
        device=device,
        load_textures=True,
        create_texture_atlas=True,
        texture_atlas_size=4,
        texture_wrap="repeat"
         )
    
    atlas = aux.texture_atlas
   
    mesh = Meshes(
        verts=[verts],
        faces=[faces.verts_idx],
        textures=TexturesAtlas(atlas=[atlas]),) 
    R, T = look_at_view_transform(dist = 0, elev = 
                              0, azim = 0) 
    cameras = FoVPerspectiveCameras(device=device, R=R, T=T)

 
    raster_settings = RasterizationSettings(
                      image_size = image_size, 
                      blur_radius = 0.0, 
                      faces_per_pixel = 1, 
                      )


    rasterizer = MeshRasterizer(
                  cameras=cameras, 
                  raster_settings=raster_settings
        )
    shader = SoftPhongShader(device , cameras = cameras)
    renderer = MeshRenderer(rasterizer, shader)
    batch_size = 4
    meshes = mesh.extend(batch_size)
    elev = torch.linspace(0, 360, batch_size)
    azim = [0,90,180,270]
    R, T = look_at_view_transform(dist = dist, elev = 0, azim = azim)
    cameras = FoVPerspectiveCameras(device=device, R=R, T=T)
    images = renderer(meshes, cameras=cameras)
    save = []
    for i in range(len(images)) :
        ims =  tensor_to_image(images[i].cpu().numpy())
        save.append(ims)
    !mkdir -p "base/base_car/"
    for i in range(len(save)) :
        save[i].save("base/base_car/"+'ID1_'+str(i)+".png")

In [87]:
def get_image_dmg(obj_filename,image_size,dist,device=device):
    verts, faces, aux = load_obj(
        obj_filename,
        device=device,
        load_textures=True,
        create_texture_atlas=True,
        texture_atlas_size=4,
        texture_wrap="repeat"
         )
    
    atlas = aux.texture_atlas
   
    mesh = Meshes(
        verts=[verts],
        faces=[faces.verts_idx],
        textures=TexturesAtlas(atlas=[atlas]),) 
    R, T = look_at_view_transform(dist = 0, elev = 
                              0, azim = 0) 
    cameras = FoVPerspectiveCameras(device=device, R=R, T=T)

 
    raster_settings = RasterizationSettings(
                      image_size = image_size, 
                      blur_radius = 0.0, 
                      faces_per_pixel = 1, 
                      )


    rasterizer = MeshRasterizer(
                  cameras=cameras, 
                  raster_settings=raster_settings
        )
    shader = SoftPhongShader(device , cameras = cameras)
    renderer = MeshRenderer(rasterizer, shader)
    batch_size = 4
    meshes = mesh.extend(batch_size)
    elev = torch.linspace(0, 360, batch_size)
    azim = [0,90,180,270]
    R, T = look_at_view_transform(dist = dist, elev = 0, azim = azim)
    cameras = FoVPerspectiveCameras(device=device, R=R, T=T)
    images = renderer(meshes, cameras=cameras)
    save = []
    for i in range(len(images)) :
        ims =  tensor_to_image(images[i].cpu().numpy())
        save.append(ims)
    !mkdir -p "dmg/car_dmg/"
    for i in range(len(save)) :
        save[i].save("dmg/car_dmg/"+'ID1_'+str(i)+".png")

In [88]:
obj1 = '/content/ID1.obj'
obj2 = '/content/ID1dmgf.obj'

In [86]:
get_image_base(obj1,1024,5)

/usr/local/lib/python3.7/dist-packages/pytorch3d/io/obj_io.py:533: UserWarning: Mtl file does not exist: /content/ID1.mtl
  warnings.warn(f"Mtl file does not exist: {f}")


In [89]:
get_image_dmg(obj2,1024,5)

/usr/local/lib/python3.7/dist-packages/pytorch3d/io/obj_io.py:533: UserWarning: Mtl file does not exist: /content/ID1dmgf.mtl
  warnings.warn(f"Mtl file does not exist: {f}")


In [91]:
car1_dataset , car1_loader = get_dataset(r'/content/base')
dmg1_dataset , dmg1_loader = get_dataset(r'/content/dmg')

In [92]:
id1b = get_features(vgg,car1_loader)
id1d = get_features(vgg,dmg1_loader)

100%|██████████| 1/1 [00:00<00:00,  3.82it/s]


In [93]:
id1 = get_euclidean(id1b,id1d)
id1

[23.755023956298828, 9.630194664001465, 0.0, 4.260119438171387]

In [95]:
get_damage(id1)

ด้านหน้าของรถยนต์มีความเสียหาย 59.38%
ด้านขวาของรถยนต์มีความเสียหาย 15.81%
ด้านหลังของรถยนต์มีความเสียหาย -19.23%
ด้านซ้ายของรถยนต์มีความเสียหาย -18.92%
